In [1]:
!pip install keras-tuner

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
fashion_mnist=keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [5]:
train_images=train_images/255.0
test_images=test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images= train_images.reshape(len(train_images),28,28,1)
test_images= test_images.reshape(len(test_images),28,28,1)

In [8]:
def build_model(hp):
  model=keras.Sequential([
                          keras.layers.Conv2D(
                              filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
                              kernel_size=hp.Choice('conv_1_kernel', values= [3,5]),
                              activation='relu',
                              input_shape=(28,28,1)
                          ),
                          keras.layers.Conv2D(
                              filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
                              kernel_size=hp.Choice('conv_2_kernel', values= [3,5]),
                              activation='relu',
                          ),
                          keras.layers.Flatten(),
                          keras.layers.Dense(
                              units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
                              activation='relu'
                          ),
                          keras.layers.Dense(10, activation='softmax')#output layer
  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [9]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameter

In [10]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5, directory='output', project_name='Mnist Fashion')

In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 6 Complete [00h 17m 22s]
val_accuracy: 0.8666666746139526

Best val_accuracy So Far: 0.903166651725769
Total elapsed time: 01h 09m 19s
INFO:tensorflow:Oracle triggered exit


In [13]:
model=tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 96)        2496      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        55360     
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                2478160   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 2,536,826
Trainable params: 2,536,826
Non-trainable params: 0
_________________________________________________________________
